In [1]:
import sys
import numpy as np
from pathlib import Path

In [2]:
if "/home/alex/qi3/hmmtuf" not in sys.path:
    sys.path.append("/home/alex/qi3/hmmtuf")

In [22]:
from db.sqlite3_db_connector import SQLiteDBConnector
from compute_engine.src.utils import count_kmers
from compute_engine.src.utils import INFO

In [23]:
# connect to the DB
db_connector = SQLiteDBConnector(db_file="../../play_ground.sqlite3")

In [24]:
MIN_REPEAT_LENGTH = 10
N_BEST_REPEATS = 5
KMER_LENGTH = 4

In [25]:
def get_max_kmers(kmers, min_count):
    
    
    # sort the dictionary
    sorted_kmers = {k: v for k, v in sorted(kmers.items(), key=lambda item: item[1], reverse=True)}
    
    top_kmers = dict()
    
    counter = 0
    for item in sorted_kmers:
        
        if counter < min_count:
            top_kmers[item] = sorted_kmers[item]
            counter += 1
        else:
            break
        
    return top_kmers

In [16]:
tbl_names = db_connector.get_table_names()

print("DB table names: {0}".format(tbl_names))

DB table names: ['distance_metric_type', 'distance_sequence_type', 'repeats', 'hmm_state_types', 'repeats_distances', 'repeats_info', 'gquads_info']


In [17]:
print(db_connector.get_table_column_names(table_name='repeats'))

[(0, 'id', 'INTEGER', 0, None, 1), (1, 'chromosome', 'TEXT', 1, None, 0), (2, 'start_idx', 'INT', 1, None, 0), (3, 'end_idx', 'INT', 1, None, 0), (4, 'repeat_seq', 'TEXT', 1, None, 0), (5, 'hmm_state_id', 'INTEGER', 1, None, 0), (6, 'gc', 'FLOAT', 0, None, 0)]


In [18]:
# repeat sequences and their state
sql = "SELECT repeats.repeat_seq,  hmm_state_types.type FROM repeats \
JOIN hmm_state_types WHERE repeats.hmm_state_id=hmm_state_types.id AND length(repeats.repeat_seq) > {0}".format(MIN_REPEAT_LENGTH)

In [19]:
repeats = db_connector.fetch_all(sql=sql)

In [20]:
print("Number of results={0}".format(len(repeats)))

Number of results=1896


In [30]:
kmers = {}

In [32]:
count = 0
for repeat in repeats:
    repeat_seq = repeat[0]
    kmers = count_kmers(sequence=repeat_seq, k=KMER_LENGTH + 4)
    top_kmers = get_max_kmers(kmers=kmers, min_count=N_BEST_REPEATS)
    count += 1
    
    print("{0} Sequence {1} state {2}".format(INFO, repeat_seq, repeat[1]))
    print("{0} Top kmers {1}".format(INFO, top_kmers))
    
    #if count > 1:
    #    break
    
    
    

INFO: Sequence CGGGGACTCTCTCTCTCTCCCGGGGCAGGTGGGGCTCTC state TUF
INFO: Top kmers {'CTCTCTCT': 3, 'TCTCTCTC': 3, 'CGGGGACT': 1, 'GGGGACTC': 1, 'GGGACTCT': 1}
INFO: Sequence ACTCCCCAATTCTTCCAGGCAGCCATCTCCCCC state NORMAL
INFO: Top kmers {'ACTCCCCA': 1, 'CTCCCCAA': 1, 'TCCCCAAT': 1, 'CCCCAATT': 1, 'CCCAATTC': 1}
INFO: Sequence GTGACGCCCCCCGACCCCTCACCCCGCTGCCGTCACACACCCCGACCCCGCTGCCGTCACTCCCCACAACCCCCCACCCCGCTGCCGTCACGCTCCGTGACCCCCGACCCCGACACCCTGCTGCCGTGATGTTCCCCCAACCCTGCACCCCGCTGCCGTGACGCCCGCCCCGACCCCGCACCCCGCTGCCGTCACGCCCCCGACCCCGCACCCCGCCGCCGTCACGCCCGACCCCTCACCCCGCTACT state DELETION
INFO: Top kmers {'CCCGACCC': 6, 'CCGACCCC': 6, 'CACCCCGC': 6, 'ACCCCGCT': 6, 'GCTGCCGT': 6}
INFO: Sequence CAGGCCAGGGAGGAGCACCACCAGGGAGGAG state TUF
INFO: Top kmers {'CCAGGGAG': 2, 'CAGGGAGG': 2, 'AGGGAGGA': 2, 'GGGAGGAG': 2, 'CAGGCCAG': 1}
INFO: Sequence CAGCGGGTCAGGGCTCTGTCAAGACTGCTGGTGAATGGCCGGCTGAGGAGGCAGCGAGG state TUF
INFO: Top kmers {'CAGCGGGT': 1, 'AGCGGGTC': 1, 'GCGGGTCA': 1, 'CGGGTCAG': 1, 'GGGTCA

INFO: Sequence TCTCTGTCTCTGACTGTGTCCC state TUF
INFO: Top kmers {'TCTCTGTC': 1, 'CTCTGTCT': 1, 'TCTGTCTC': 1, 'CTGTCTCT': 1, 'TGTCTCTG': 1}
INFO: Sequence TGGTGGGGAGGAGGCTGGGGTGGGGAGGAGGCTAGAT state TUF
INFO: Top kmers {'GGTGGGGA': 2, 'GTGGGGAG': 2, 'TGGGGAGG': 2, 'GGGGAGGA': 2, 'GGGAGGAG': 2}
INFO: Sequence CCCACCTCTACACTGAACCACGCCCTTTCCCCCACT state TUF
INFO: Top kmers {'CCCACCTC': 1, 'CCACCTCT': 1, 'CACCTCTA': 1, 'ACCTCTAC': 1, 'CCTCTACA': 1}
INFO: Sequence CACCTCTGTACTGAACCACGCCCCCTTCCGTCACCCC state DELETION
INFO: Top kmers {'CACCTCTG': 1, 'ACCTCTGT': 1, 'CCTCTGTA': 1, 'CTCTGTAC': 1, 'TCTGTACT': 1}
INFO: Sequence TCCCTCCCCCTCCCCATC state DELETION
INFO: Top kmers {'CCCTCCCC': 2, 'TCCCTCCC': 1, 'CCTCCCCC': 1, 'CTCCCCCT': 1, 'TCCCCCTC': 1}
INFO: Sequence CTCCTGCCTCAGCCTCCCGAGTAGCTGGGATTACAGGCGCCCACCACCATGCCCAGCTAATTTTTTGTATTTTTAGTAGAAACGGGGTTTCACCATGTTGGCCAAGATGGTCTCAAACTCCTGACCTCAGGTGACCCACCCACCTCGGCCTCCCAAAGTGCTGGGATGACAGACGCAAGCCACCACACCCAGCCTATGTGAGGAACTCTTAAAACAACCACCAACAGGGGGGACA

INFO: Sequence TTATACATATATATATATATATATACACAAACAAACAAAAATGTCTGTGCAAGCAAAAAAAAAAAAACACATTAGTTGCCACTTGTACGATGGAGAAACTATAAAACAGGTAATATTTTCAAAGATCAAAACACGAAAAAAAAAAAACAAGATATAGAGATGGAAAAATTTGTAAAGGGATTTAAAAACTAGCTTCTTCAAACTCAACATCAGACTTATTAAAATATAATTTAGAAAATCCAGAAACATAAAAACATGCCATAGTGTTTACGTGCTCTTTTAAAATCATAAAAAAAAACTACAAAAAAAAGATGTATCAACTGTATTCAAACTTTAGCACTTGAACGGAAAAACACAAAAATTTCCTATGTATTTGGGGGAAAATTATATACTTTGTACCAAAGACATATACAACTGATATGCCAAACTGTCAGTCTAAGTATAGTTTAAAACAATGTACATTAAACTCTAAAAACTTATGAATTAAAAGCTGAAAAAAGAAAAAAAAAAAATTCAGCAAGAAGGAAATTACTGAAAACTTCAAACATTGTTTCTGTTTGGGAAAAATTATCTACAGAACCAGCACCCACACAAACAAATATTATTTAACTATACCTCGACTACAAAGCCACAAAAGATAAATTGAAAACTATTTAAATCTATTTTATTTCTGCACTTATTGTAGGAAATTTTAATATATTTCATTTTAGTAAGCTATTGATAAAATAGTTTTTGACTTTGAAAATTAAAATGTTTATTTAGCTTATTGTAGTATACTTCCACCAGACAACAAAATAGATTATTT state TUF
INFO: Top kmers {'AAAAAAAA': 19, 'ATATATAT': 6, 'TATATATA': 6, 'ACAAACAA': 3, 'CAAACAAA': 3}
INFO: Sequence CCCCCCTCCTCCTCCTCATCTCCTA state DELETION
INFO: Top kmers {'CC